# Lista 6

Modelo CER de ativos correlacionados
1. Importar dados do Yahoo contendo a série histórica dos últimos 12 meses das
ações da Petrobrás, Vale, Ambev e Banco do Brasil.
2. Estime os parâmetros do modelo CER de cada um dos ativos
3. Verifique a sua normalidade
4. Calcule a matriz de correlação empirica entre esses ativos.
5. Compare os resultados da simulação MC do CER correlacionado com os respectivos dados históricos.
6. Estime os valores dos ativos em Dezembro/2024

## 1. Importar Dados do Yahoo Finance

- Utilizando a biblioteca quantmod para obter os dados históricos dos últimos 12 meses das ações da Petrobrás (PETR4.SA), Vale (VALE3.SA), Ambev (ABEV3.SA) e Banco do Brasil (BBAS3.SA).

### Instalar e carregar as bibliotecas necessárias

- [quantmod](https://www.quantmod.com)
  - Uso: Para importar dados financeiros históricos de várias fontes, incluindo Yahoo Finance. Utilizamos essa biblioteca para obter os preços ajustados das ações da Petrobrás, Vale, Ambev e Banco do Brasil.
- [tseries](https://uribo.github.io/rpkg_showcase/date_and_time/tseries.html)
  - Uso: Para realizar testes estatísticos. Utilizamos essa biblioteca para aplicar o teste de Jarque-Bera e verificar a normalidade dos retornos dos ativos.
- [PerformanceAnalytics](https://github.com/braverock/PerformanceAnalytics)
  - Uso: Para realizar análises de desempenho de investimentos. Utilizamos essa biblioteca para calcular retornos e realizar outras operações relacionadas a desempenho de ativos financeiros.
- [MASS](https://www.stats.ox.ac.uk/pub/MASS4/)
  - Uso: Para gerar amostras de distribuições multivariadas. Utilizamos essa biblioteca para simular retornos futuros dos ativos utilizando a matriz de covariância estimada.

In [2]:
install.packages("quantmod")
install.packages("tseries")
install.packages("PerformanceAnalytics")
install.packages("MASS")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(quantmod)
library(tseries)
library(PerformanceAnalytics)
library(MASS)

Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘PerformanceAnalytics’


The following object is masked from ‘package:graphics’:

    legend




### Definir os símbolos dos ativos

In [4]:
simbolos <- c("PETR4.SA", "VALE3.SA", "ABEV3.SA", "BBAS3.SA")

### Obter os dados dos últimos 12 meses

In [5]:
getSymbols(simbolos, src = "yahoo", from = Sys.Date() - 365, to = Sys.Date())

[1] "PETR4.SA" "VALE3.SA" "ABEV3.SA" "BBAS3.SA"

### Fechar os preços ajustados

In [6]:
precos <- do.call(merge, lapply(simbolos, function(sym) Ad(get(sym))))

### Visualizar os dados

In [7]:
head(precos)

           PETR4.SA.Adjusted VALE3.SA.Adjusted ABEV3.SA.Adjusted
2023-05-29          21.64717          60.56482          14.02682
2023-05-30          21.40385          59.13878          13.76073
2023-05-31          21.18487          58.70638          13.69421
2023-06-01          21.86615          59.94840          13.53265
2023-06-02          22.04459          62.50605          13.74172
2023-06-05          22.27979          61.99084          13.69421
           BBAS3.SA.Adjusted
2023-05-29          19.55960
2023-05-30          19.72730
2023-05-31          19.69200
2023-06-01          19.74496
2023-06-02          20.33999
2023-06-05          20.46729

## 2. Estimar os Parâmetros do Modelo CER de Cada Ativo

- Primeiro, vamos calcular os retornos logarítmicos dos preços ajustados. Em seguida, estimamos a média e a variância desses retornos.

### Calcular retornos logarítmicos

In [8]:
retornos <- na.omit(diff(log(precos)))

### Estimar os parâmetros do modelo CER

In [9]:
medias <- colMeans(retornos)
cov_matrix <- cov(retornos)


### Visualizar os parâmetros estimados

In [10]:
medias
cov_matrix

PETR4.SA.Adjusted VALE3.SA.Adjusted ABEV3.SA.Adjusted BBAS3.SA.Adjusted 
     0.0021273152      0.0002911060     -0.0006759394      0.0013171226

,PETR4.SA.Adjusted,VALE3.SA.Adjusted,ABEV3.SA.Adjusted,BBAS3.SA.Adjusted
PETR4.SA.Adjusted,3.527068e-04,5.952787e-05,1.315664e-06,6.958803e-05
VALE3.SA.Adjusted,5.952787e-05,2.262752e-04,1.703152e-05,3.442413e-05
ABEV3.SA.Adjusted,1.315664e-06,1.703152e-05,1.510995e-04,5.889585e-05
BBAS3.SA.Adjusted,6.958803e-05,3.442413e-05,5.889585e-05,1.686199e-04


## 3. Verificar a Normalidade

- Vamos utilizar o teste de Jarque-Bera para verificar a normalidade dos retornos.

### Testar normalidade dos retornos

In [11]:
testes_normalidade <- apply(retornos, 2, jarque.bera.test)

### Visualizar os resultados dos testes de normalidade

In [12]:
testes_normalidade

$PETR4.SA.Adjusted

	Jarque Bera Test

data:  newX[, i]
X-squared = 198.38, df = 2, p-value < 2.2e-16


$VALE3.SA.Adjusted

	Jarque Bera Test

data:  newX[, i]
X-squared = 25.493, df = 2, p-value = 2.912e-06


$ABEV3.SA.Adjusted

	Jarque Bera Test

data:  newX[, i]
X-squared = 124.22, df = 2, p-value < 2.2e-16


$BBAS3.SA.Adjusted

	Jarque Bera Test

data:  newX[, i]
X-squared = 18.798, df = 2, p-value = 8.282e-05



## 4. Calcular a Matriz de Correlação Empírica

- Vamos calcular a matriz de correlação entre os retornos dos ativos.

## Calcular matriz de correlação

In [13]:
cor_matrix <- cor(retornos)

### Visualizar matriz de correlação

In [14]:
cor_matrix

,PETR4.SA.Adjusted,VALE3.SA.Adjusted,ABEV3.SA.Adjusted,BBAS3.SA.Adjusted
PETR4.SA.Adjusted,1.000000000,0.21071476,0.005699104,0.2853470
VALE3.SA.Adjusted,0.210714757,1.00000000,0.092109314,0.1762342
ABEV3.SA.Adjusted,0.005699104,0.09210931,1.000000000,0.3689767
BBAS3.SA.Adjusted,0.285346965,0.17623417,0.368976659,1.0000000


## 5. Comparar Resultados da Simulação MC do CER Correlacionado com Dados Históricos

- Vamos simular os retornos dos ativos utilizando a matriz de correlação e comparar os resultados com os dados históricos.

### Número de simulações

In [15]:
numero_simulacoes <- 10000
numeros_ativos <- ncol(retornos)
numero_periodos <- nrow(retornos)

### Simular retornos futuros usando Monte Carlo

In [16]:
set.seed(123)
retornos_simulados <- mvrnorm(numero_simulacoes * numero_periodos,
                              mu = medias, Sigma = cov_matrix)
retornos_simulados <- matrix(retornos_simulados,
                             ncol = numeros_ativos, byrow = TRUE)

### Comparar os retornos simulados com os retornos históricos

In [17]:
media_simulada <- colMeans(retornos_simulados)
simulada_cov_matrix <- cov(retornos_simulados)

### Visualizar comparação

In [18]:
list(
  MediaHistorica = medias,
  MediaSimulada = media_simulada,
  CovarianciaHistorica = cov_matrix,
  CovarianciaSimulada = simulada_cov_matrix
)

$MediaHistorica
PETR4.SA.Adjusted VALE3.SA.Adjusted ABEV3.SA.Adjusted BBAS3.SA.Adjusted 
     0.0021273152      0.0002911060     -0.0006759394      0.0013171226 

$MediaSimulada
[1] 0.0007605974 0.0007669516 0.0007553278 0.0007604589

$CovarianciaHistorica
                  PETR4.SA.Adjusted VALE3.SA.Adjusted ABEV3.SA.Adjusted
PETR4.SA.Adjusted      3.527068e-04      5.952787e-05      1.315664e-06
VALE3.SA.Adjusted      5.952787e-05      2.262752e-04      1.703152e-05
ABEV3.SA.Adjusted      1.315664e-06      1.703152e-05      1.510995e-04
BBAS3.SA.Adjusted      6.958803e-05      3.442413e-05      5.889585e-05
                  BBAS3.SA.Adjusted
PETR4.SA.Adjusted      6.958803e-05
VALE3.SA.Adjusted      3.442413e-05
ABEV3.SA.Adjusted      5.889585e-05
BBAS3.SA.Adjusted      1.686199e-04

$CovarianciaSimulada
             [,1]         [,2]         [,3]         [,4]
[1,] 2.257448e-04 1.251105e-06 1.166645e-06 8.933478e-07
[2,] 1.251105e-06 2.259429e-04 1.020872e-06 1.115110e-06
[3,] 1.166645e-06 1.020872e-06 2.257653e-04 8.558650e-07
[4,] 8.933478e-07 1.115110e-06 8.558650e-07 2.257816e-04

## 6. Estimar Valores dos Ativos em Dezembro/2024

- Vamos projetar os preços dos ativos para dezembro de 2024 utilizando os parâmetros estimados.

### Período de projeção (restante do ano)

In [19]:
periodo_projetado <- as.integer(difftime(as.Date("2024-12-31"), Sys.Date(), units = "days"))

### Estimar valores futuros usando a média e covariância dos retornos

In [21]:
precos_simulados <- matrix(0, nrow = numero_simulacoes, ncol = numeros_ativos)
precos_atuais <- tail(precos, 1)

In [22]:
for (i in 1:numero_simulacoes) {
  retornos_futuros <- mvrnorm(periodo_projetado, mu = medias / 252, Sigma = cov_matrix / 252)
  retornos_futuros_cum <- apply(retornos_futuros, 2, cumsum)
  precos_simulados[i, ] <- precos_atuais * exp(tail(retornos_futuros_cum, 1))
}

### Estimar os valores médios futuros em dezembro/2024

In [23]:
precos_futuros_estimados <- colMeans(precos_simulados)

### Visualizar os valores estimados

In [24]:
data.frame(
  Ativos = simbolos,
  PrecoAtuais = as.numeric(precos_atuais),
  Preco_Futuro_Estimado = precos_futuros_estimados
)

Ativos,PrecoAtuais,Preco_Futuro_Estimado
<chr>,<dbl>,<dbl>
PETR4.SA,36.61,36.68369
VALE3.SA,65.08,65.09555
ABEV3.SA,11.87,11.86465
BBAS3.SA,27.08,27.11357


## Resumo

Este script em R segue os passos para importar dados históricos, estimar parâmetros do modelo CER, verificar a normalidade, calcular a matriz de correlação empírica, comparar simulações Monte Carlo com dados históricos e projetar preços futuros dos ativos.